<H1 style = "font-family:Arial;font-weight:bold;color:purple;font-size:30px;"><strong>Capstone Project – Part 1 with YOLOv8</strong></H1>
This notebook trains YOLOv8 on your converted dataset.

In [1]:
# Step 0: Install Ultralytics (YOLOv8)
#!pip install ultralytics --quiet
from ultralytics import YOLO
print("Ultralytics installed.")


Ultralytics installed.


In [2]:
import pandas as pd
import os
from PIL import Image
import os
import shutil
import random
from sklearn.model_selection import train_test_split

In [33]:
# Paths (update these to your dataset location)
images_dir = r"C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets\Capstone 1\Part 1\Images"#directory where Images are saved
labels_dir = "labels.csv"#labels csv file name
output_dir = r"C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets\Capstone 1\Part 1\dataset"#directory to create train and val directory

# Train/Val split ratio
val_ratio = 0.2  


# Load the labels from labels.csv

labels_df = pd.read_csv('labels.csv', sep=',', header=None)
labels_df.columns = ['image_id', 'class', 'x_min', 'y_min', 'x_max', 'y_max']

# Adjust the image IDs in the dataframe
labels_df['image_id'] = labels_df['image_id'].apply(lambda x: f"{x:08d}")

In [34]:
# setting numeric labels to class type
classes = labels_df['class'].unique().tolist()
class_to_id = {cls: idx for idx, cls in enumerate(classes)}
labels_df['class'] = labels_df['class'].map(class_to_id)

In [35]:
class_to_id

{'pickup_truck': 0,
 'car': 1,
 'articulated_truck': 2,
 'bus': 3,
 'motorized_vehicle': 4,
 'work_van': 5,
 'single_unit_truck': 6,
 'pedestrian': 7,
 'bicycle': 8,
 'non-motorized_vehicle': 9,
 'motorcycle': 10}

In [16]:
# Save classes.txt
with open(os.path.join(output_dir, "classes.txt"), "w") as f:
    f.write("\n".join(classes))

In [17]:
# ------------------ SPLIT DATASET ------------------
all_images = labels_df['image_id'].unique()
train_images, val_images = train_test_split(all_images, test_size=val_ratio, random_state=42)
splits = {'train': train_images, 'val': val_images}

In [18]:
for split in splits:
    os.makedirs(os.path.join(output_dir, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, split, "labels"), exist_ok=True)

In [19]:
# ------------------ PROCESS IMAGES ------------------
possible_extensions = possible_extensions = ['.jpg', '.jpeg', '.png']  # extensions to check
missing_images = []

for split_name, split_images in splits.items():
    for img_id in split_images:
        # Detect correct extension
        img_name = None
        for ext in possible_extensions:
            if os.path.exists(os.path.join(images_dir, f"{img_id}{ext}")):
                img_name = f"{img_id}{ext}"
                break

        if img_name is None:
            missing_images.append(img_id)
            continue

        img_path = os.path.join(images_dir, img_name)

        # Copy image
        shutil.copy(img_path, os.path.join(output_dir, split_name, "images", img_name))

        # Filter labels for this image
        group = labels_df[labels_df['image_id'] == img_id]

        # Generate YOLO label lines
        yolo_lines = []
        w, h = Image.open(img_path).size
        for _, row in group.iterrows():
            class_id = int(row['class'])
            x_center = ((row['x_min'] + row['x_max']) / 2) / w
            y_center = ((row['y_min'] + row['y_max']) / 2) / h
            width = (row['x_max'] - row['x_min']) / w
            height = (row['y_max'] - row['y_min']) / h
            yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # Save YOLO label file (empty if no objects)
        txt_file = os.path.join(output_dir, split_name, "labels", f"{img_id}.txt")
        with open(txt_file, 'w') as f:
            f.write("\n".join(yolo_lines))

# ------------------ LOG MISSING IMAGES ------------------
if missing_images:
    with open(os.path.join(output_dir, "missing_images.txt"), "w") as f:
        f.write("\n".join(missing_images))
    print(f"⚠️ {len(missing_images)} images not found. See missing_images.txt")

print("✅ Dataset processed successfully! Train/Val folders and YOLO labels are ready.")

⚠️ 104374 images not found. See missing_images.txt
✅ Dataset processed successfully! Train/Val folders and YOLO labels are ready.


In [20]:
# Step 1: Paths and sanity checks
DATA_YAML = "data.yaml"
print("Using:", DATA_YAML)
import os
assert os.path.exists(DATA_YAML), "data.yaml not found!"


Using: data.yaml


In [21]:
# Step 2: Train a small YOLOv8 model (nano) for a quick start
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # you can change to yolov8s.pt, yolov8m.pt, etc.
results = model.train(
    data=DATA_YAML,
    epochs=20,      # increase this for better results
    imgsz=640,
    batch=16
)


New https://pypi.org/project/ultralytics/8.3.184 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.182  Python-3.13.5 torch-2.8.0+cpu CPU (AMD Ryzen 5 7535HS with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, opt

train: Scanning C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets\Capstone 1\Part 1\data

val: Fast image access  (ping: 0.10.0 ms, read: 3.11.4 MB/s, size: 27.0 KB)



C:\Users\Padmavathi\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets\Capstone 1\Part 1\datase
C:\Users\Padmavathi\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20         0G     0.8861     0.6387     0.9352         35        640: 100%|██████████| 281/281 [16:50<00:00,  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [01:04

                   all       1134       3675      0.649      0.627       0.66      0.482



20 epochs completed in 6.460 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 6.2MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.182  Python-3.13.5 torch-2.8.0+cpu CPU (AMD Ryzen 5 7535HS with Radeon Graphics)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:54


                   all       1134       3675      0.658      0.624       0.66      0.482
          pickup_truck        296        413      0.772      0.811      0.869      0.709
                   car        941       2493      0.841      0.882       0.92      0.672
     articulated_truck        102        115      0.803      0.791      0.847      0.652
                   bus         98        107      0.881      0.935      0.957      0.874
     motorized_vehicle        203        254       0.55      0.313      0.337      0.195
              work_van         91         95      0.623      0.611      0.618      0.497
     single_unit_truck         65         70      0.512      0.414      0.468      0.335
            pedestrian         37         66      0.606      0.242       0.38      0.185
               bicycle         22         27      0.562      0.815      0.771      0.502
 non-motorized_vehicle         18         18      0.401      0.278      0.239      0.152
            motorcycl

In [22]:
# Step 3: Evaluate on the validation set
metrics = model.val()
print(metrics)


Ultralytics 8.3.182  Python-3.13.5 torch-2.8.0+cpu CPU (AMD Ryzen 5 7535HS with Radeon Graphics)
Model summary (fused): 72 layers, 3,007,793 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.20.1 ms, read: 50.528.8 MB/s, size: 25.7 KB)


val: Scanning C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets\Capstone 1\Part 1\datase
C:\Users\Padmavathi\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 71/71 [00:55


                   all       1134       3675      0.658      0.624       0.66      0.482
          pickup_truck        296        413      0.772      0.811      0.869      0.709
                   car        941       2493      0.841      0.882       0.92      0.672
     articulated_truck        102        115      0.803      0.791      0.847      0.652
                   bus         98        107      0.881      0.935      0.957      0.874
     motorized_vehicle        203        254       0.55      0.313      0.337      0.195
              work_van         91         95      0.623      0.611      0.618      0.497
     single_unit_truck         65         70      0.512      0.414      0.468      0.335
            pedestrian         37         66      0.606      0.242       0.38      0.185
               bicycle         22         27      0.562      0.815      0.771      0.502
 non-motorized_vehicle         18         18      0.401      0.278      0.239      0.152
            motorcycl

In [23]:
# Step 4: Inference on a few validation images and save results
pred_results = model.predict(source="C:/Users/Padmavathi/OneDrive/Desktop/python/Capstone Project/Capstone 1/Datasets/Capstone 1/Part 1/dataset/val/images", save=True, conf=0.25, max_det=300)
print("Predictions saved under runs/detect/")



WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1134 C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets\Capstone 1\Part 1\dataset\val\images\00000003.jpg: 448x640 2 cars, 103.3ms
image 2/1134 C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets\Capstone 1\Part 1\dataset\val\images\00000023.jpg: 448x640 2 pickup_trucks, 2 cars, 1 work_van, 51.5ms
image 3/1134 C:\Users\Padmavathi\OneDrive\Desktop\python\Capstone Project\Capstone 1\Datasets